# Imports

In [33]:
from typing import Dict, Text
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

# Read Data

In [2]:
consumer_data=pd.read_csv("data/consumer_transanctions.csv")
platform_data=pd.read_csv("data/platform_content.csv")

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
consumer_data

,event_timestamp,interaction_type,item_id,consumer_id,consumer_session_id,consumer_device_info,consumer_location,country
0,1465413032,content_watched,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,content_watched,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,content_watched,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,content_followed,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,content_watched,-7820640624231356730,-445337111692715325,561148 1178424124714,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
72307,1485190425,content_liked,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR
72308,1485190425,content_watched,-5813211845057621660,102305705598210278,5527770709392883642,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
72309,1485190072,content_watched,-1999468346928419252,-9196668942822132778,-8300596454915870873,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR
72310,1485190434,content_watched,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR


# Analysis

In [5]:
consumer_data.interaction_type.value_counts()

content_watched         61086
content_liked            5745
content_saved            2463
content_commented_on     1611
content_followed         1407
Name: interaction_type, dtype: int64

In [6]:
# We can give rating to the rows based on the interaction_type 
# after much consideration we decided to give the following ratings to each field ranging 1-5
# content_watched          3   watched should be neutral as we dont know if user liked that content or not
# content_liked            5   liked shold be max rating as user have liked means user already watched and liked it
# content_saved            2   saved should be less than watched
# content_commented_on     4   if user commented anything means user watched and we dont know what kind of comment user has posted that means it should be less than liked.
# content_followed         5   followed means user watched and liked the content

In [7]:
consumer_data.item_id.nunique()

2987

In [8]:
consumer_data.consumer_id.nunique()

1895

In [17]:
consumer_data.country.value_counts()

BR    51396
US     4657
KR      239
CA      226
JP      144
AU      138
GB       22
DE       19
IE       14
IS       13
ZZ       11
SG       11
AR        7
PT        6
ES        3
IN        3
MY        2
IT        2
CN        1
NL        1
CL        1
CO        1
CH        1
Name: country, dtype: int64

In [18]:
# based on the above data distribution from consumer table - we only need interaction_type, item_id & consumer_id

In [7]:
platform_data

,event_timestamp,interaction_type,item_id,producer_id,producer_session_id,producer_device_info,producer_location,producer_country,item_type,item_url,title,text_description,language
0,1459192779,content_pulled_out,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,content_present,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,content_present,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,content_present,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,content_present,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3117,1487946604,content_present,9213260650272029784,3609194402293569455,7144190892417579456,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,SP,BR,HTML,https://startupi.com.br/2017/02/liga-ventures-...,"Conheça a Liga IoT, plataforma de inovação abe...","A Liga Ventures, aceleradora de startups espec...",pt
3118,1487947067,content_present,-3295913657316686039,6960073744377754728,-8193630595542572738,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3...,GA,US,HTML,https://thenextweb.com/apps/2017/02/14/amazon-...,Amazon takes on Skype and GoToMeeting with its...,"Amazon has launched Chime, a video conferencin...",en
3119,1488223224,content_present,3618271604906293310,1908339160857512799,-183341653743161643,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_0...,SP,BR,HTML,https://code.org/about/2016,Code.org 2016 Annual Report,"February 9, 2017 - We begin each year with a l...",en
3120,1488300719,content_present,6607431762270322325,-1393866732742189886,2367029511384577082,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,MG,BR,HTML,https://www.bloomberg.com/news/articles/2017-0...,JPMorgan Software Does in Seconds What Took La...,"At JPMorgan Chase & Co., a learning machine is...",en


In [9]:
platform_data.interaction_type.value_counts()

content_present       3047
content_pulled_out      75
Name: interaction_type, dtype: int64

In [10]:
platform_data.item_id.nunique()

3057

In [11]:
platform_data.producer_id.nunique()

252

In [12]:
platform_data.item_type.value_counts()

HTML     3101
VIDEO      11
RICH       10
Name: item_type, dtype: int64

In [13]:
platform_data.language.value_counts()

en    2264
pt     850
la       4
es       2
ja       2
Name: language, dtype: int64

In [14]:
# based on the above data distribution from platform table - we only need interaction_type & item_id some other columns can be used as additional features to improve the results

# Processing

### identify items not to recommend based on content_pulled_out

In [4]:
removed_items=list(platform_data[platform_data.interaction_type=="content_pulled_out"]["item_id"])

In [5]:
len(set(removed_items))

74

In [6]:
# as per the use case we are remove other language content also
len(set(list(platform_data[platform_data.language!="en"]["item_id"])))

839

In [7]:
removed_items=removed_items+list(platform_data[platform_data.language!="en"]["item_id"])
len(removed_items)

933

In [8]:
removed_items=list(set(removed_items))
len(removed_items)

891

In [9]:
platform_data.shape

(3122, 13)

In [10]:
removed_index1=[]
for i in range(len(platform_data)):
    if platform_data["item_id"][i] in removed_items:
        removed_index1.append(i)
removed_index2=[]
for i in range(len(consumer_data)):
    if consumer_data["item_id"][i] in removed_items:
        removed_index2.append(i)

In [11]:
len(removed_index1),len(removed_index2)

(956, 25464)

In [12]:
platform_data=platform_data.drop(removed_index1)
platform_data.reset_index(drop=True)
platform_data.shape

(2166, 13)

In [13]:
consumer_data.shape

(72312, 8)

In [14]:
consumer_data=consumer_data.drop(removed_index2)
consumer_data.reset_index(drop=True)
consumer_data.shape

(46848, 8)

In [ ]:
# prepare data for training 

In [16]:
consumer_data=consumer_data[["interaction_type","item_id","consumer_id"]]

In [17]:
consumer_data["rating"]=0
consumer_data.loc[consumer_data["interaction_type"]=="content_watched","rating"]=3
consumer_data.loc[consumer_data["interaction_type"]=="content_liked","rating"]=5
consumer_data.loc[consumer_data["interaction_type"]=="content_saved","rating"]=2
consumer_data.loc[consumer_data["interaction_type"]=="content_commented_on","rating"]=4
consumer_data.loc[consumer_data["interaction_type"]=="content_followed","rating"]=5

In [18]:
#convert columns valueees into str as they aree unique valuees
consumer_data["consumer_id"]=consumer_data["consumer_id"].astype("str")
consumer_data["item_id"]=consumer_data["item_id"].astype("str")

In [19]:
consumer_data

,interaction_type,item_id,consumer_id,rating
0,content_watched,-3499919498720038879,-8845298781299428018,3
1,content_watched,8890720798209849691,-1032019229384696495,3
4,content_watched,-7820640624231356730,-445337111692715325,3
7,content_watched,-1492913151930215984,4254153380739593270,3
9,content_watched,3064370296170038610,3609194402293569455,3
...,...,...,...,...
72305,content_watched,569574447134368517,-5230721907253934520,3
72306,content_watched,5484061377044071389,3609194402293569455,3
72308,content_watched,-5813211845057621660,102305705598210278,3
72309,content_watched,-1999468346928419252,-9196668942822132778,3


In [20]:
consumer_data.rating.value_counts()

3    39643
5     4329
2     1845
4     1031
Name: rating, dtype: int64

In [36]:
#load data into tf dataset

In [21]:
users=tf.data.Dataset.from_tensor_slices(consumer_data[["item_id","consumer_id"]])

In [22]:
users

<TensorSliceDataset element_spec=TensorSpec(shape=(2,), dtype=tf.string, name=None)>

In [23]:
users=users.map(lambda x: {
    "consumer_id": x[1],
    "item_id": x[0]
})

In [24]:
users

<MapDataset element_spec={'consumer_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'item_id': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [25]:
for i in users.take(2):
    print(i)

{'consumer_id': <tf.Tensor: shape=(), dtype=string, numpy=b'-8845298781299428018'>, 'item_id': <tf.Tensor: shape=(), dtype=string, numpy=b'-3499919498720038879'>}
{'consumer_id': <tf.Tensor: shape=(), dtype=string, numpy=b'-1032019229384696495'>, 'item_id': <tf.Tensor: shape=(), dtype=string, numpy=b'8890720798209849691'>}


In [27]:
items=tf.data.Dataset.from_tensor_slices(platform_data["item_id"].astype("str"))
items

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [28]:
for i in items.take(2):
    print(i)

tf.Tensor(b'-4110354420726924665', shape=(), dtype=string)
tf.Tensor(b'-7292285110016212249', shape=(), dtype=string)


# Model Building

# Approach 1

In [29]:
#Build vocabularies to convert consumer_id and item_id titles into integer indices for embedding layers:

In [30]:
consumer_id_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
consumer_id_vocabulary.adapt(users.map(lambda x: x["consumer_id"]))

item_id_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
item_id_vocabulary.adapt(items)

In [34]:
class Model(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      item_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.item_model = item_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["consumer_id"])
    item_embeddings = self.item_model(features["item_id"])

    return self.task(user_embeddings, item_embeddings)

In [35]:
# Define user and item models.
user_model = tf.keras.Sequential([
    consumer_id_vocabulary,
    tf.keras.layers.Embedding(consumer_id_vocabulary.vocab_size(), 64)
])
item_model = tf.keras.Sequential([
    item_id_vocabulary,
    tf.keras.layers.Embedding(item_id_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    items.batch(128).map(item_model)
  )
)

In [36]:
# Create a retrieval model.
model = Model(user_model, item_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 10 epochs.
model.fit(users.batch(2048), epochs=20)


Epoch 1/20
23/23 [==============================] - 14s 463ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0102 - factorized_top_k/top_5_categorical_accuracy: 0.0324 - factorized_top_k/top_10_categorical_accuracy: 0.0471 - factorized_top_k/top_50_categorical_accuracy: 0.0942 - factorized_top_k/top_100_categorical_accuracy: 0.1387 - loss: 16459.7829 - regularization_loss: 0.0000e+00 - total_loss: 16459.7829
Epoch 2/20
23/23 [==============================] - 12s 501ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0187 - factorized_top_k/top_5_categorical_accuracy: 0.0823 - factorized_top_k/top_10_categorical_accuracy: 0.1256 - factorized_top_k/top_50_categorical_accuracy: 0.2769 - factorized_top_k/top_100_categorical_accuracy: 0.3711 - loss: 14096.4101 - regularization_loss: 0.0000e+00 - total_loss: 14096.4101
Epoch 3/20
23/23 [==============================] - 10s 449ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0266 - factorized_top_k/top_5_categorical_accu

23/23 [==============================] - 11s 484ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0286 - factorized_top_k/top_5_categorical_accuracy: 0.1517 - factorized_top_k/top_10_categorical_accuracy: 0.2510 - factorized_top_k/top_50_categorical_accuracy: 0.6076 - factorized_top_k/top_100_categorical_accuracy: 0.7665 - loss: 10473.3640 - regularization_loss: 0.0000e+00 - total_loss: 10473.3640


In [37]:
# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

In [38]:
index.index_from_dataset(
    items.batch(100).map(lambda title: (title, model.item_model(title))))

In [39]:
#pick random user to get recommendation
index(np.array(["344280948527967603"]))

(<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[16.216024, 15.306022, 10.076336,  9.738819,  9.720582,  9.671001,
          9.344862,  9.325453,  9.308453,  9.131302]], dtype=float32)>,
 <tf.Tensor: shape=(1, 10), dtype=string, numpy=
 array([[b'521382866757596943', b'3569727790804487273',
         b'-1897853393557630253', b'-6874540813378776198',
         b'-6119620726358216329', b'-3864069617285644895',
         b'-8278375716950339547', b'-5829979886567267504',
         b'-8939172344092554931', b'1542516532533301214']], dtype=object)>)

In [41]:
def title_name(item_id):
    return platform_data[platform_data["item_id"]==item_id]["title"].values[0]

In [42]:
title_name(-6192959589652486637)

"Google's official Project Wing logo revealed"

In [43]:
# Get some recommendations.
_, titles = index(np.array(["8845298781299428018"]))
print(f"Top 10 recommendations :\n")
count=0
flag=False
for i in titles:
    for j in i:
        if int(j) in removed_items:
            continue
        print(title_name(int(j)))
        count-=-1
        if count==10:
            flag=True
            break
    if flag:
        break

Top 10 recommendations :

Java Microservices: The Cake is a Lie but You Can't Ignore It | Takipi Blog
The Gold Mine: A Novel of Lean Turnaround
Test and deploy to Google App Engine with the new Maven and Gradle plugins
Use Docker to build, test and push your Artifacts
Java Tools and Technologies Landscape Report 2016 | zeroturnaround.com
Microsoft is taking its cues from Facebook as it totally shakes up Office
How IoT security can benefit from machine learning
Google, Amazon and the upcoming battle over AI Assistants
Get your apps and games ready for space with Google Play | Android Developers Blog
Top 10 GCP sessions from Google I/O 2016


# Approach 2 retrieval

In [176]:
tf.random.set_seed(42)
shuffled = users.shuffle(100000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(60000)
test = shuffled.skip(60000)

In [179]:
yitems = items.batch(1_000)
xuser = users.batch(1_000).map(lambda x: x["consumer_id"])

unique_items = np.unique(np.concatenate(list(yitems)))
unique_users = np.unique(np.concatenate(list(xuser)))

for i in unique_items[:10]:
    print(title_name(int(i)))

Google unleashes DeepMind on energy-hungry datacenter, cutting cooling bill by 40 percent
Indústria 4.0: desafios e oportunidades
12 JavaScript Hacks
Australian Bitcoin Entrepreneur Launches Robo-Advice Platform
React Native v0.32.0-rc.0 released
Para entender o Dia Internacional contra a Homofobia
Apple launches Safari Technology Preview, a browser for developers including experimental web features
Hero unveils a new home gadget to help you track and dispense pills
Google Drive Is Scary-Smart at Searching Your Images
Facial recognition systems stumble when confronted with million-face database


In [180]:
embedding_dimension = 32

In [181]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_users, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_users) + 1, embedding_dimension)
])

In [182]:
candidate_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_items, mask_token=None),
  tf.keras.layers.Embedding(len(unique_items) + 1, embedding_dimension)
])

In [183]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=items.batch(128).map(candidate_model)
)
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [188]:
class Model2(tfrs.Model):

  def __init__(self, user_model, candidate_model):
    super().__init__()
    self.candidate_model: tf.keras.Model = candidate_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["consumer_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.candidate_model(features["item_id"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

In [ ]:
# same alternative model with different lib below

In [187]:
# class Model2(tf.keras.Model):

#   def __init__(self, user_model, candidate_model):
#     super().__init__()
#     self.candidate_model: tf.keras.Model = candidate_model
#     self.user_model: tf.keras.Model = user_model
#     self.task: tf.keras.layers.Layer = task

#   def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

#     # Set up a gradient tape to record gradients.
#     with tf.GradientTape() as tape:

#       # Loss computation.
#       user_embeddings = self.user_model(features["consumer_id"])
#       positive_movie_embeddings = self.candidate_model(features["item_id"])
#       loss = self.task(user_embeddings, positive_movie_embeddings)

#       # Handle regularization losses as well.
#       regularization_loss = sum(self.losses)

#       total_loss = loss + regularization_loss

#     gradients = tape.gradient(total_loss, self.trainable_variables)
#     self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

#     metrics = {metric.name: metric.result() for metric in self.metrics}
#     metrics["loss"] = loss
#     metrics["regularization_loss"] = regularization_loss
#     metrics["total_loss"] = total_loss

#     return metrics

#   def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

#     # Loss computation.
#     user_embeddings = self.user_model(features["consumer_id"])
#     positive_movie_embeddings = self.candidate_model(features["item_id"])
#     loss = self.task(user_embeddings, positive_movie_embeddings)

#     # Handle regularization losses as well.
#     regularization_loss = sum(self.losses)

#     total_loss = loss + regularization_loss

#     metrics = {metric.name: metric.result() for metric in self.metrics}
#     metrics["loss"] = loss
#     metrics["regularization_loss"] = regularization_loss
#     metrics["total_loss"] = total_loss

#     return metrics

In [189]:
model = Model2(user_model, candidate_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [190]:
cached_train = train.shuffle(100_00).batch(1024).cache()
cached_test = test.batch(4096).cache()

In [191]:
model.fit(cached_train, epochs=20)

Epoch 1/20
59/59 [==============================] - 18s 261ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0039 - factorized_top_k/top_5_categorical_accuracy: 0.0571 - factorized_top_k/top_10_categorical_accuracy: 0.0860 - factorized_top_k/top_50_categorical_accuracy: 0.1990 - factorized_top_k/top_100_categorical_accuracy: 0.2773 - loss: 6642.5167 - regularization_loss: 0.0000e+00 - total_loss: 6642.5167
Epoch 2/20
59/59 [==============================] - 15s 248ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0127 - factorized_top_k/top_5_categorical_accuracy: 0.1265 - factorized_top_k/top_10_categorical_accuracy: 0.1855 - factorized_top_k/top_50_categorical_accuracy: 0.3853 - factorized_top_k/top_100_categorical_accuracy: 0.4974 - loss: 5820.4797 - regularization_loss: 0.0000e+00 - total_loss: 5820.4797
Epoch 3/20
59/59 [==============================] - 14s 241ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0138 - factorized_top_k/top_5_categorical_accuracy

In [192]:
model.evaluate(cached_test, return_dict=True)

4/4 [==============================] - 5s 652ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0209 - factorized_top_k/top_5_categorical_accuracy: 0.1000 - factorized_top_k/top_10_categorical_accuracy: 0.1576 - factorized_top_k/top_50_categorical_accuracy: 0.3549 - factorized_top_k/top_100_categorical_accuracy: 0.4637 - loss: 17874.9723 - regularization_loss: 0.0000e+00 - total_loss: 17874.9723


{'factorized_top_k/top_1_categorical_accuracy': 0.020873943343758583,
 'factorized_top_k/top_5_categorical_accuracy': 0.09998375922441483,
 'factorized_top_k/top_10_categorical_accuracy': 0.15756985545158386,
 'factorized_top_k/top_50_categorical_accuracy': 0.354938268661499,
 'factorized_top_k/top_100_categorical_accuracy': 0.4636939465999603,
 'loss': 73.8243179321289,
 'regularization_loss': 0,
 'total_loss': 73.8243179321289}

In [193]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends content out of the entire platform dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((items.batch(100), items.batch(100).map(model.candidate_model)))
)


In [194]:
# Get some recommendations.
_, titles = index(np.array(["8845298781299428018"]))
print(f"Top 10 recommendations :\n")
count=0
flag=False
for i in titles:
    for j in i:
        if int(j) in removed_items:
            continue
        print(title_name(int(j)))
        count-=-1
        if count==10:
            flag=True
            break
    if flag:
        break

Top 10 recommendations :

Research at Google
Nespresso
IBM Watson e a revolução pela inteligência artificial | Udacity
Vision API - Image Content Analysis
Udacity abre código de simulador de carro autônomo para quem quiser testar
Descubra os 10 apps mais importantes para o brasileiro
BDD Best Practices and Guidelines - Testing Excellence
Fabric is Joining Google
Services 
Front in BH 2016


# Approach 3 Ranking

In [ ]:
# this approach we will use ratings as additional feature

In [213]:
consumer_data

,interaction_type,item_id,consumer_id,rating
0,content_watched,-3499919498720038879,-8845298781299428018,3
1,content_watched,8890720798209849691,-1032019229384696495,3
2,content_watched,310515487419366995,-1130272294246983140,3
3,content_followed,310515487419366995,344280948527967603,5
4,content_watched,-7820640624231356730,-445337111692715325,3
...,...,...,...,...
72307,content_liked,-6590819806697898649,-9016528795238256703,5
72308,content_watched,-5813211845057621660,102305705598210278,3
72309,content_watched,-1999468346928419252,-9196668942822132778,3
72310,content_watched,-6590819806697898649,-9016528795238256703,3


In [44]:
consumer_data["rating"]=consumer_data["rating"].astype("str")

In [198]:
# rating_label=tf.data.Dataset.from_tensor_slices(consumer_data["rating"])

In [45]:
users=tf.data.Dataset.from_tensor_slices(consumer_data[["item_id","consumer_id","rating"]])
users=users.map(lambda x: {
    "consumer_id": x[1],
    "item_id": x[0],
    "rating": int(x[2])
})

In [46]:
users

<MapDataset element_spec={'consumer_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'item_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'rating': TensorSpec(shape=(), dtype=tf.int32, name=None)}>

In [47]:
train = users.take(65000)
test = users.skip(65000)

In [48]:
yitems = users.batch(1_000).map(lambda x: x["item_id"])
xuser = users.batch(1_000).map(lambda x: x["consumer_id"])

unique_items = np.unique(np.concatenate(list(yitems)))
unique_users = np.unique(np.concatenate(list(xuser)))

In [247]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_users, mask_token=None),
      tf.keras.layers.Embedding(len(unique_users) + 1, embedding_dimension)
    ])

    
    self.candidate_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_items, mask_token=None),
      tf.keras.layers.Embedding(len(unique_items) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      tf.keras.layers.Dense(32, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, item_id = inputs

    user_embedding = self.user_embeddings(user_id)
    candidate_embeddings = self.candidate_embeddings(item_id)

    return self.ratings(tf.concat([user_embedding, candidate_embeddings], axis=1))

In [248]:
#This model takes user ids and movie titles, and outputs a predicted rating:
RankingModel()((["-8845298781299428018"], ["8890720798209849691"]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.02246787]], dtype=float32)>

In [249]:
class Model3(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["consumer_id"], features["item_id"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [250]:
model = Model3()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [251]:
cached_train = train.shuffle(100000).batch(2048).cache()
cached_test = test.batch(1000).cache()

In [252]:
model.fit(cached_train, epochs=20)

Epoch 1/20
32/32 [==============================] - 3s 20ms/step - root_mean_squared_error: 1.1597 - loss: 1.3095 - regularization_loss: 0.0000e+00 - total_loss: 1.3095
Epoch 2/20
32/32 [==============================] - 0s 15ms/step - root_mean_squared_error: 0.6434 - loss: 0.4141 - regularization_loss: 0.0000e+00 - total_loss: 0.4141
Epoch 3/20
32/32 [==============================] - 0s 15ms/step - root_mean_squared_error: 0.6413 - loss: 0.4113 - regularization_loss: 0.0000e+00 - total_loss: 0.4113
Epoch 4/20
32/32 [==============================] - 0s 14ms/step - root_mean_squared_error: 0.6388 - loss: 0.4081 - regularization_loss: 0.0000e+00 - total_loss: 0.4081
Epoch 5/20
32/32 [==============================] - 0s 15ms/step - root_mean_squared_error: 0.6363 - loss: 0.4050 - regularization_loss: 0.0000e+00 - total_loss: 0.4050
Epoch 6/20
32/32 [==============================] - 1s 19ms/step - root_mean_squared_error: 0.6343 - loss: 0.4023 - regularization_loss: 0.0000e+00 - total

In [253]:
model.evaluate(cached_test, return_dict=True)

8/8 [==============================] - 3s 31ms/step - root_mean_squared_error: 0.6462 - loss: 0.4400 - regularization_loss: 0.0000e+00 - total_loss: 0.4400


{'root_mean_squared_error': 0.6462299227714539,
 'loss': 0.5371559262275696,
 'regularization_loss': 0,
 'total_loss': 0.5371559262275696}

In [254]:
#Testing the ranking model

In [255]:
consumer_data

,interaction_type,item_id,consumer_id,rating
0,content_watched,-3499919498720038879,-8845298781299428018,3
1,content_watched,8890720798209849691,-1032019229384696495,3
2,content_watched,310515487419366995,-1130272294246983140,3
3,content_followed,310515487419366995,344280948527967603,5
4,content_watched,-7820640624231356730,-445337111692715325,3
...,...,...,...,...
72307,content_liked,-6590819806697898649,-9016528795238256703,5
72308,content_watched,-5813211845057621660,102305705598210278,3
72309,content_watched,-1999468346928419252,-9196668942822132778,3
72310,content_watched,-6590819806697898649,-9016528795238256703,3


In [256]:
test_ratings = {}
test_id = ["-1999468346928419252", "310515487419366995", "-3499919498720038879	"]
for iid in test_id:
  test_ratings[iid] = model({
      "consumer_id": np.array(["-8845298781299428018"]),
      "item_id": np.array([iid])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
310515487419366995: [[3.518589]]
-3499919498720038879	: [[3.4054434]]
-1999468346928419252: [[3.3590615]]


In [257]:
#based on the above results it looks like model works :)

# Approach 4 

In [49]:
#Multitask recommender / Hybrid

In [51]:
users,items

(<MapDataset element_spec={'consumer_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'item_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'rating': TensorSpec(shape=(), dtype=tf.int32, name=None)}>,
 <TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>)

In [53]:
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = users.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(65000)
test = shuffled.skip(65000)

items_ids = items.batch(1_000)
user_ids = users.batch(1_000).map(lambda x: x["consumer_id"])

unique_items = np.unique(np.concatenate(list(items_ids)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [58]:
class Model4(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32

    # User and candidate_model.
    self.candidate_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_items, mask_token=None),
      tf.keras.layers.Embedding(len(unique_items) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A small model to take in user and candidate embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(8, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=items.batch(128).map(self.candidate_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["consumer_id"])
    # And pick out the item features and pass them into the candidate_model.
    item_embeddings = self.candidate_model(features["item_id"])

    return (
        user_embeddings,
        item_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, item_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("rating")

    user_embeddings, item_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, item_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

### Approach 4.1

In [59]:
model4_1 = Model4(rating_weight=1.0, retrieval_weight=0.0)
model4_1.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [76]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [62]:
model4_1.fit(cached_train, epochs=20)

Epoch 1/20
6/6 [==============================] - 16s 2s/step - root_mean_squared_error: 1.1389 - factorized_top_k/top_1_categorical_accuracy: 3.8422e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0018 - factorized_top_k/top_10_categorical_accuracy: 0.0042 - factorized_top_k/top_50_categorical_accuracy: 0.0216 - factorized_top_k/top_100_categorical_accuracy: 0.0439 - loss: 1.1324 - regularization_loss: 0.0000e+00 - total_loss: 1.1324
Epoch 2/20
6/6 [==============================] - 15s 2s/step - root_mean_squared_error: 0.6364 - factorized_top_k/top_1_categorical_accuracy: 2.3480e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0018 - factorized_top_k/top_10_categorical_accuracy: 0.0042 - factorized_top_k/top_50_categorical_accuracy: 0.0216 - factorized_top_k/top_100_categorical_accuracy: 0.0440 - loss: 0.4045 - regularization_loss: 0.0000e+00 - total_loss: 0.4045
Epoch 3/20
6/6 [==============================] - 14s 2s/step - root_mean_squared_error: 0.6447 - factorized_top

6/6 [==============================] - 18s 3s/step - root_mean_squared_error: 0.6360 - factorized_top_k/top_1_categorical_accuracy: 2.3480e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0020 - factorized_top_k/top_10_categorical_accuracy: 0.0041 - factorized_top_k/top_50_categorical_accuracy: 0.0212 - factorized_top_k/top_100_categorical_accuracy: 0.0439 - loss: 0.4034 - regularization_loss: 0.0000e+00 - total_loss: 0.4034
Epoch 20/20
6/6 [==============================] - 19s 3s/step - root_mean_squared_error: 0.6359 - factorized_top_k/top_1_categorical_accuracy: 2.7749e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0021 - factorized_top_k/top_10_categorical_accuracy: 0.0041 - factorized_top_k/top_50_categorical_accuracy: 0.0212 - factorized_top_k/top_100_categorical_accuracy: 0.0442 - loss: 0.4033 - regularization_loss: 0.0000e+00 - total_loss: 0.4033


In [80]:
metrics = model4_1.evaluate(cached_test, return_dict=True,verbose=1)

print(f"Retrieval top-10 accuracy: {metrics['factorized_top_k/top_10_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

6/6 [==============================] - 8s 1s/step - root_mean_squared_error: 0.6357 - factorized_top_k/top_1_categorical_accuracy: 4.2691e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0021 - factorized_top_k/top_10_categorical_accuracy: 0.0039 - factorized_top_k/top_50_categorical_accuracy: 0.0211 - factorized_top_k/top_100_categorical_accuracy: 0.0441 - loss: 0.4044 - regularization_loss: 0.0000e+00 - total_loss: 0.4044
Retrieval top-10 accuracy: 0.004.
Ranking RMSE: 0.636.


### Approach 4.2

In [81]:
model4_2 = Model4(rating_weight=0.0, retrieval_weight=1.0)
model4_2.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [84]:
model4_2.fit(cached_train, epochs=20)

Epoch 1/20
6/6 [==============================] - 11s 2s/step - root_mean_squared_error: 3.3047 - factorized_top_k/top_1_categorical_accuracy: 0.0496 - factorized_top_k/top_5_categorical_accuracy: 0.1772 - factorized_top_k/top_10_categorical_accuracy: 0.2469 - factorized_top_k/top_50_categorical_accuracy: 0.4691 - factorized_top_k/top_100_categorical_accuracy: 0.5857 - loss: 57513.1998 - regularization_loss: 0.0000e+00 - total_loss: 57513.1998
Epoch 2/20
6/6 [==============================] - 12s 2s/step - root_mean_squared_error: 3.3137 - factorized_top_k/top_1_categorical_accuracy: 0.0561 - factorized_top_k/top_5_categorical_accuracy: 0.1920 - factorized_top_k/top_10_categorical_accuracy: 0.2709 - factorized_top_k/top_50_categorical_accuracy: 0.5080 - factorized_top_k/top_100_categorical_accuracy: 0.6269 - loss: 55094.4548 - regularization_loss: 0.0000e+00 - total_loss: 55094.4548
Epoch 3/20
6/6 [==============================] - 11s 2s/step - root_mean_squared_error: 3.3197 - factor

6/6 [==============================] - 14s 2s/step - root_mean_squared_error: 3.3468 - factorized_top_k/top_1_categorical_accuracy: 0.0483 - factorized_top_k/top_5_categorical_accuracy: 0.2126 - factorized_top_k/top_10_categorical_accuracy: 0.3189 - factorized_top_k/top_50_categorical_accuracy: 0.6124 - factorized_top_k/top_100_categorical_accuracy: 0.7405 - loss: 47778.9743 - regularization_loss: 0.0000e+00 - total_loss: 47778.9743
Epoch 20/20
6/6 [==============================] - 14s 2s/step - root_mean_squared_error: 3.3476 - factorized_top_k/top_1_categorical_accuracy: 0.0483 - factorized_top_k/top_5_categorical_accuracy: 0.2125 - factorized_top_k/top_10_categorical_accuracy: 0.3192 - factorized_top_k/top_50_categorical_accuracy: 0.6137 - factorized_top_k/top_100_categorical_accuracy: 0.7423 - loss: 47679.2054 - regularization_loss: 0.0000e+00 - total_loss: 47679.2054


In [85]:
metrics = model4_2.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-10 accuracy: {metrics['factorized_top_k/top_10_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

6/6 [==============================] - 13s 2s/step - root_mean_squared_error: 3.3480 - factorized_top_k/top_1_categorical_accuracy: 0.0524 - factorized_top_k/top_5_categorical_accuracy: 0.2241 - factorized_top_k/top_10_categorical_accuracy: 0.3321 - factorized_top_k/top_50_categorical_accuracy: 0.6232 - factorized_top_k/top_100_categorical_accuracy: 0.7505 - loss: 47202.4554 - regularization_loss: 0.0000e+00 - total_loss: 47202.4554
Retrieval top-10 accuracy: 0.332.
Ranking RMSE: 3.348.


### Approach 4.3

In [86]:
model4_3 = Model4(rating_weight=1.0, retrieval_weight=1.0)
model4_3.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [87]:
model4_3.fit(cached_train, epochs=20)

Epoch 1/20
6/6 [==============================] - 15s 2s/step - root_mean_squared_error: 3.1216 - factorized_top_k/top_1_categorical_accuracy: 0.0205 - factorized_top_k/top_5_categorical_accuracy: 0.0689 - factorized_top_k/top_10_categorical_accuracy: 0.0987 - factorized_top_k/top_50_categorical_accuracy: 0.2008 - factorized_top_k/top_100_categorical_accuracy: 0.2674 - loss: 66489.1339 - regularization_loss: 0.0000e+00 - total_loss: 66489.1339
Epoch 2/20
6/6 [==============================] - 12s 2s/step - root_mean_squared_error: 1.7437 - factorized_top_k/top_1_categorical_accuracy: 0.0394 - factorized_top_k/top_5_categorical_accuracy: 0.1424 - factorized_top_k/top_10_categorical_accuracy: 0.1997 - factorized_top_k/top_50_categorical_accuracy: 0.3908 - factorized_top_k/top_100_categorical_accuracy: 0.5088 - loss: 61476.1172 - regularization_loss: 0.0000e+00 - total_loss: 61476.1172
Epoch 3/20
6/6 [==============================] - 12s 2s/step - root_mean_squared_error: 0.7190 - factor

6/6 [==============================] - 13s 2s/step - root_mean_squared_error: 0.6204 - factorized_top_k/top_1_categorical_accuracy: 0.0487 - factorized_top_k/top_5_categorical_accuracy: 0.2109 - factorized_top_k/top_10_categorical_accuracy: 0.3159 - factorized_top_k/top_50_categorical_accuracy: 0.6070 - factorized_top_k/top_100_categorical_accuracy: 0.7350 - loss: 48017.0876 - regularization_loss: 0.0000e+00 - total_loss: 48017.0876
Epoch 20/20
6/6 [==============================] - 12s 2s/step - root_mean_squared_error: 0.6200 - factorized_top_k/top_1_categorical_accuracy: 0.0491 - factorized_top_k/top_5_categorical_accuracy: 0.2110 - factorized_top_k/top_10_categorical_accuracy: 0.3171 - factorized_top_k/top_50_categorical_accuracy: 0.6090 - factorized_top_k/top_100_categorical_accuracy: 0.7373 - loss: 47899.6702 - regularization_loss: 0.0000e+00 - total_loss: 47899.6702


In [92]:
metrics = model4_3.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-10 accuracy: {metrics['factorized_top_k/top_10_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

6/6 [==============================] - 11s 2s/step - root_mean_squared_error: 0.6194 - factorized_top_k/top_1_categorical_accuracy: 0.0542 - factorized_top_k/top_5_categorical_accuracy: 0.2208 - factorized_top_k/top_10_categorical_accuracy: 0.3287 - factorized_top_k/top_50_categorical_accuracy: 0.6192 - factorized_top_k/top_100_categorical_accuracy: 0.7458 - loss: 47399.3404 - regularization_loss: 0.0000e+00 - total_loss: 47399.3404
Retrieval top-10 accuracy: 0.329.
Ranking RMSE: 0.619.


In [ ]:
# All the above models have been evaluated based on the top 10 accurarcy and its RMSE scores.
### Why Why RMSE is better than MSE?
# It's a metric for determining how close a fitted line is to the real data points. The error rate by the square 
# root of MSE is called RMSE (Root Mean Squared Error). RMSE is a better measure of fit than a correlation coefficient
# since it can be immediately translated into dimension units.

### Why top_10_categorical_accuracy
# In essence, accuracy is able to tell if the recommender system is able to predict those items that you have already rated
# or interacted with, thus recommender systems which optimize accuracy will naturally place those items at the top of a user's
# list.

In [94]:
# based on the different experiments and model approaches. I decided that approach 4.3 is best as its a mixture of ranking & 
# retrival both models, a hybrid with many haperparameters to tune. We can further improve these results by hyperparmeter tuning
# 


In [95]:
# next steps 
# We can build cross network recomendation 
# or try efficient serving
# listwise ranking may also help here